In [35]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
            labeled_data = [(item.get("data"), item.get("label")) for item in data]
            return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error: Unable to decode JSON file '{filename}': {e}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")

    return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [spell.candidates(word) for word in words_to_check]
    return corrected_words, suggestions


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker(language="en")
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Initialize an empty list to keep track of misclassified samples
    misclassified_samples = []
    misclassification_threshold = 5
    epochs = 0

    # Initialize empty DataFrames to store all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df is None:
            return

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        model, vectorizer = train_model(all_X_train, all_y_train)

        # Check for misclassified samples
        words_to_check = ["motday", "hgost", "mney"]
        predictions = model.predict(vectorizer.transform(words_to_check))
        misclassified_samples.extend(
            [
                word
                for word, prediction in zip(words_to_check, predictions)
                if not prediction
            ]
        )

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save custom dictionary incrementally
    new_vocabulary = vectorizer.get_feature_names_out()
    if os.path.exists(custom_dictionary_filename):
        with open(custom_dictionary_filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        new_vocabulary = existing_vocabulary.union(new_vocabulary)
    with open(custom_dictionary_filename, "w") as file:
        json.dump(list(new_vocabulary), file)

    # Save misclassified samples
    with open(misclassified_samples_filename, "w") as file:
        json.dump(misclassified_samples, file)

    # Check spelling and print corrected words and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, suggestion_list in zip(corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {word}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
            labeled_data = [(item.get("data"), item.get("label")) for item in data]
            return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error: Unable to decode JSON file '{filename}': {e}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")

    return None


def save_custom_dictionary(filename, new_vocabulary):
    """Save custom dictionary as JSON."""
    custom_dictionary = set()

    # Load existing custom dictionary if it exists
    if os.path.exists(filename):
        with open(filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        # Union existing vocabulary with new vocabulary
        custom_dictionary.update(existing_vocabulary)

    # Update custom dictionary with new vocabulary
    custom_dictionary.update(new_vocabulary)

    # Save the updated custom dictionary
    with open(filename, "w") as file:
        json.dump(list(custom_dictionary), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions."""
    # Convert the list of words to a pandas Series
    words_series = pd.Series(words_to_check)

    # Use apply method to apply the correction function to each word
    corrected_words_series = words_series.apply(spell.correction)

    # Use apply method to apply the candidates function to each word
    suggestions_series = words_series.apply(spell.candidates)

    # Convert the corrected words and suggestions back to lists
    corrected_words = corrected_words_series.tolist()
    suggestions = suggestions_series.tolist()

    return corrected_words, suggestions


def fit_vectorizer(X_train, vectorizer=None):
    """Fit or update a CountVectorizer."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)
    return vectorizer


def train_model(X_train_vectorized, y_train):
    """Train a Naive Bayes model."""
    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)
    return model


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker(language="en")
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Initialize an empty list to keep track of misclassified samples
    misclassified_samples = []
    misclassification_threshold = 5
    epochs = 0

    # Initialize empty DataFrames to store all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df is None:
            return

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        vectorizer = fit_vectorizer(all_X_train)

        X_train_vectorized = vectorizer.transform(all_X_train)

        model = train_model(X_train_vectorized, all_y_train)

        # Check for misclassified samples
        words_to_check = ["kng", "hgost", "mney", "hllo", "temle","lon", "aler"]
        predictions = model.predict(vectorizer.transform(words_to_check))
        misclassified_samples.extend(
            [
                word
                for word, prediction in zip(words_to_check, predictions)
                if not prediction
            ]
        )

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save custom dictionary incrementally
    new_vocabulary = vectorizer.get_feature_names_out()
    if os.path.exists(custom_dictionary_filename):
        with open(custom_dictionary_filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        new_vocabulary = existing_vocabulary.union(new_vocabulary)
    with open(custom_dictionary_filename, "w") as file:
        json.dump(list(new_vocabulary), file)

    # Save misclassified samples
    with open(misclassified_samples_filename, "w") as file:
        json.dump(misclassified_samples, file)

    # Check spelling and print corrected words and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, suggestion_list in zip(corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {word}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()

Original: king, Corrected: king, Suggestions: {'ing', 'keg', 'ng', 'king'}
Original: ghost, Corrected: ghost, Suggestions: {'ghost', 'host'}
Original: money, Corrected: money, Suggestions: {'money'}
Original: hello, Corrected: hello, Suggestions: {'halo', 'hallo', 'hollo', 'hello'}
Original: temple, Corrected: temple, Suggestions: {'temple'}
Original: on, Corrected: on, Suggestions: {'lo', 'yon', 'son', 'on', 'ion', 'ln', 'lion', 'ton', 'lop', 'long', 'hon', 'lone', 'loin', 'loan', 'non', 'lob', 'won', 'con', 'loon', 'eon', 'lox', 'don', 'lorn', 'low', 'lot', 'loo', 'log'}
Original: alert, Corrected: alert, Suggestions: {'taler', 'alder', 'abler', 'aller', 'baler', 'alter', 'alert', 'ales', 'alar', 'aver', 'ale', 'paler', 'alee', 'haler'}
Training loop finished.


In [6]:
4 + (3 % 5)

7

In [28]:
print(3 % 5)
print(10 % 5)
print(7 % 5)
print(5 % 7)
print(5 % 7)
print(4 + (3 % 5))

3
0
2
5
5
7


In [14]:
from typing import  List
def threeSum(nums: List[int]) -> List[List[int]]:
    if len(nums) < 3: # Requires 3 for a pair of 3
        return []
    elif len(nums) == 3 and sum(nums) == 0: # Naive case; check sum of 3 elements
        return [nums]

    nums = sorted(nums) # O(nlogn)

    solutions = []
    i = 0
    while i < len(nums) - 2:                       # Minus 2 because need 3 elements
        if i > 0 and nums[i] == nums[i-1]:         # Skip any duplicates
            i += 1
            continue
  
        # Solve a 2 sum
        j = i + 1
        k = len(nums) - 1
        target = 0 - nums[i]
  
        while j < k:
            # Skip duplicates
            if j > i + 1 and nums[j] == nums[j-1]:
                j += 1
                continue
  
            # Skip duplicates
            if k < len(nums)-1 and nums[k] == nums[k+1]:
                k -= 1
                continue

            jkSum = nums[j] + nums[k]
            if jkSum == target: # Found pair
                solutions.append([nums[i], nums[j], nums[k]])

            if jkSum > target:
                k -= 1
            else:
                j += 1 # Advances from solution too

        i += 1
  
    return solutions

In [18]:
threeSum(nums=[-1, 0, 1, 2, -1, -4, 6, -5])

[[-5, -1, 6], [-1, -1, 2], [-1, 0, 1]]

In [26]:
print(7 / 4)
print(7 // 4)
print(7 % 4)

minutes = 645
hours = minutes // 60
print(hours)

print(6//4)
print(-6//4)


1.75
1
3
10
1
-2


In [34]:
print(76%77)
print(76//77)
print(76/77)

76
0
0.987012987012987


In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from spellchecker import SpellChecker
import os

# Import machine learning libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def load_labeled_data(filename):
    # Function to load labeled data from JSON file (existing implementation)

def fit_vectorizer(X_train, vectorizer=None):
    # Function to fit or update CountVectorizer (existing implementation)

def train_naive_bayes_model(X_train_vectorized, y_train):
    # Function to train Naive Bayes model (existing implementation)

def train_machine_learning_model(X_train, y_train, vocab_size):
    # Tokenize words and pad sequences
    tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
    tokenizer.fit_on_texts(X_train)
    X_train_sequences = tokenizer.texts_to_sequences(X_train)
    X_train_padded = pad_sequences(X_train_sequences)

    # Define the model architecture
    model = Sequential([
        Embedding(vocab_size, 16, input_length=X_train_padded.shape[1]),
        LSTM(64),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_padded, y_train, epochs=10, batch_size=32)
    
    return model

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
    )

    # Fit vectorizer (existing implementation)
    vectorizer = fit_vectorizer(X_train)

    # Train Naive Bayes model (existing implementation)
    X_train_vectorized = vectorizer.transform(X_train)
    model = train_naive_bayes_model(X_train_vectorized, y_train)

    # Train machine learning model
    vocab_size = 10000  # Adjust as needed
    machine_learning_model = train_machine_learning_model(X_train, y_train, vocab_size)

    # Rest of the main function...
